In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#load data
import json
import pandas as pd

data_path = 'backup/instagram_tagged_posts_20241229_195451.json'
sentiment_data_path = 'result-transformer/comments_result_df.csv'

# Open and read the JSON file
with open(data_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

sentiment_df = pd.read_csv(sentiment_data_path, sep=',', encoding='utf-8')

In [40]:
post_data = []

for i in range(len(data)):
    username = data[i]['post_link'].split('/')[3]
    post_data.append([username, data[i]['post_link'], data[i]['likes_count'], len(data[i]['comments']), data[i]['shares_count']])

post_data_df = pd.DataFrame(post_data, columns=["username", "post_link", "likes_count", "comments_count", "shares_count"])

In [42]:
post_data_df.to_csv('post_data_df.csv', index=False)

In [44]:
sentiment_df['post_username'] = ''
# sentiment_df['likes'] = 0

In [61]:
id, j = 0, 0

for i in range(len(data)):
    comment_count = len(data[i]['comments'])
    username = data[j]['post_link'].split('/')[3]

    if comment_count != 0:
        # Assign post_id to the range of rows
        sentiment_df.loc[id:id + comment_count - 1, 'post_username'] = username
        id += comment_count
        j += 1
    else:
        j += 1
        continue


In [62]:
sentiment_df

,Comment,Positive,Neutral,Negative,Sentiment,post_username
0,👏👏👏,0.999363,0.000110,0.000527,positive,jonyrahardja
1,@nila_riska rahasia😂😂😂,0.000523,0.000243,0.999234,negative,jonyrahardja
2,@jonyrahardja kamsia ko,0.013217,0.020829,0.965954,negative,jonyrahardja
3,@nila_riska enak bebeknya😂😂😂,0.994066,0.000267,0.005668,positive,jonyrahardja
4,Alamatnya dimn ini?,0.000279,0.000461,0.999260,negative,jonyrahardja
...,...,...,...,...,...,...
1268,NaN,NaN,NaN,NaN,NaN,
1270,NaN,NaN,NaN,NaN,NaN,
1271,NaN,NaN,NaN,NaN,NaN,
1274,NaN,NaN,NaN,NaN,NaN,


In [47]:
# sentiment_df = sentiment_df.drop(columns=['likes'])

In [63]:
sentiment_df.to_csv('sentiment_df.csv', index=False)

In [8]:
posts_comments = []

for i in range(len(data)) :
    post_link = data[i]['post_link']
    username = data[i]['post_link'].split('/')[3]
    likes = data[i]['likes_count']
    comments_count = len(data[i]['comments'])

    posts_comments.append({'post_link': post_link, 
                           'username': username,
                           'likes': likes,
                           'comments_count': comments_count,
                           })
    
    posts_comments[i]['comments'] = []
    for j in range(len(data[i]['comments'])):
        comment_username = data[i]['comments'][j]['username']
        comment = sentiment_df['Comment'][j]
        comment_label = sentiment_df['Sentiment'][j]
        
        posts_comments[i]['comments'].append({
                                                'username': comment_username,
                                                'comment': comment,
                                                'label': comment_label,
                                            })


In [13]:
posts_comments[4]

{'post_link': 'https://www.instagram.com/jonyrahardja/reel/DDrYIDHv27g/',
 'username': 'jonyrahardja',
 'likes': 449,
 'comments_count': 63,
 'comments': [{'username': 'ambarinis9',
   'comment': '👏👏👏',
   'label': 'positive'},
  {'username': 'ambarinis9',
   'comment': '@nila_riska rahasia😂😂😂',
   'label': 'negative'},
  {'username': 'ambarinis9',
   'comment': '@jonyrahardja kamsia ko',
   'label': 'negative'},
  {'username': 'ambarinis9',
   'comment': '@nila_riska enak bebeknya😂😂😂',
   'label': 'positive'},
  {'username': 'krist_ajjaahhh',
   'comment': 'Alamatnya dimn ini?',
   'label': 'negative'},
  {'username': 'krist_ajjaahhh',
   'comment': '@ambarinis9 opo iyoo say 🤭🤭',
   'label': 'neutral'},
  {'username': 'ambarinis9',
   'comment': '@krist_ajjaahhh ho oh👏👏👏enak rasanya',
   'label': 'positive'},
  {'username': 'krist_ajjaahhh',
   'comment': '@ambarinis9 👍👍',
   'label': 'positive'},
  {'username': 'mas_andhy23',
   'comment': 'Rekomend & enak 👍',
   'label': 'positive'}

In [15]:
with open("data.json", "w", encoding="utf-8") as json_file:
    json.dump(posts_comments, json_file, ensure_ascii=False, indent=4)